In [7]:
import torch
from torchvision import models, transforms
from PIL import Image
from google.colab import drive
drive.mount('/content/drive')

WEIGHTS_PATH = "/content/drive/MyDrive/pneumonia_detection_vgg/best_vgg16.pth"
IMAGE_PATH = "/content/drive/MyDrive/chest_xray/test/PNEUMONIA/person135_bacteria_646.jpeg"

ARCH = "vgg16"

CLASS_NAMES = ["NORMAL", "PNEUMONIA"]

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using:", DEVICE)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using: cpu


In [5]:
model = models.vgg16(weights=None)

model.classifier[-1] = torch.nn.Linear(model.classifier[-1].in_features, len(CLASS_NAMES))

state_dict = torch.load(WEIGHTS_PATH, map_location=DEVICE)
model.load_state_dict(state_dict)
model.to(DEVICE)
model.eval()
print(" Model loaded and ready.")


 Model loaded and ready.


In [8]:

tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

img = Image.open(IMAGE_PATH).convert("RGB")
x = tfms(img).unsqueeze(0).to(DEVICE)

with torch.no_grad():
    outputs = model(x)
    probs = torch.softmax(outputs, dim=1)[0].cpu().numpy()

for cls, p in zip(CLASS_NAMES, probs):
    print(f"{cls}: {p:.3f}")
print("Predicted class:", CLASS_NAMES[probs.argmax()])


NORMAL: 0.008
PNEUMONIA: 0.992
Predicted class: PNEUMONIA


In [21]:
import torch
from torchvision import models, transforms
from PIL import Image


WEIGHTS_PATH = "/content/drive/MyDrive/pneumonia_detection_resnet/best_resnet50.pth"
IMAGE_PATH = "/content/drive/MyDrive/chest_xray/test/PNEUMONIA/person88_bacteria_437.jpeg"

ARCH = "resnet50"
CLASS_NAMES = ["NORMAL", "PNEUMONIA"]

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)


Using device: cpu


In [22]:
model = models.resnet50(weights=None)

model.fc = torch.nn.Linear(model.fc.in_features, len(CLASS_NAMES))

state_dict = torch.load(WEIGHTS_PATH, map_location=DEVICE)
model.load_state_dict(state_dict)
model.to(DEVICE)
model.eval()

print(f"Loaded {ARCH} model with weights from:", WEIGHTS_PATH)


Loaded resnet50 model with weights from: /content/drive/MyDrive/pneumonia_detection_resnet/best_resnet50.pth


In [23]:
tfms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])
img = Image.open(IMAGE_PATH).convert("RGB")
x = tfms(img).unsqueeze(0).to(DEVICE)
with torch.no_grad():
    outputs = model(x)
    probs = torch.softmax(outputs, dim=1)[0].cpu().numpy()
for cls, p in zip(CLASS_NAMES, probs):
    print(f"{cls}: {p:.3f}")
print("Predicted class:", CLASS_NAMES[probs.argmax()])


NORMAL: 0.006
PNEUMONIA: 0.994
Predicted class: PNEUMONIA
